Instalamos libreria de spotify

In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 7.2 MB/s eta 0:00:00


Importamos librerias comunes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

#Conexión y prueba

Creamos la conexión con la api de spotify y hacemos una consulta de prueba para ver si funciona. Por cuestiones de privacidad de quitan las credenciales.

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id='',
    client_secret=''
))

track = sp.search(q='track:"To Begin Again" artist:"Ingrid Michaelson;ZAYN"', type='track')
release_date = track['tracks']['items'][0]['album']['release_date']
print(release_date)

2021-03-17


Lectura del dataset original. Este se utilizó para comenzar el proceso.

In [ ]:
df = pd.read_csv('dataset_spotify.csv')
df.head()

Lectura del progreso alcanzado, esto se hizo para tener un respaldo en caso de que el código se rompiera o ciclara, ya que debido a la gran cantidad de datos se llegaba a un límite de solicitudes.

In [ ]:
df = pd.read_csv('progress.csv')
df.tail()

/tmp/ipython-input-1345241370.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('progress.csv')


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,release_date
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.235,...,1,0.0422,0.640,0.928,0.0863,0.0339,125.995,5,world-music,NaN
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.117,...,0,0.0401,0.994,0.976,0.1050,0.0350,85.239,4,world-music,NaN
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.329,...,0,0.0420,0.867,0.000,0.0839,0.7430,132.378,4,world-music,NaN
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.506,...,1,0.0297,0.381,0.000,0.2700,0.4130,135.960,4,world-music,NaN
113999,113999,2hETkH7cOfqmz3LqZDHZf5,Cesária Evora,Miss Perfumado,Barbincor,22,241826,False,0.526,0.487,...,0,0.0725,0.681,0.000,0.0893,0.7080,79.198,4,world-music,NaN


#Consulta V1

Esta fue la primera versión de consulta que utilizamos para obtener el año de cada canción, sin embargo, solamente funcionó para los primeros 10,000 datos aproximadamente, ya que después de eso se tenían problemas por cantidad y tasa de solicitudes.

In [ ]:
import time
import pandas as pd
import spotipy

# Revisar si existe la columna 'release_date'
if 'release_date' not in df.columns:
    df['release_date'] = ""

for i, row in df.iterrows():
    # En caso de haber progreso, revisamos si esa fila ya tiene fecha, para no repetir consultas
    if not pd.isna(df.at[i, 'release_date']) and df.at[i, 'release_date'] != "":
        continue

    query = f'track:"{row["track_name"]}" artist:"{row["artists"]}"'
    try:
        result = sp.search(q=query, type='track', limit=1)
        if result['tracks']['items']:
            df.at[i, 'release_date'] = result['tracks']['items'][0]['album']['release_date']
    except spotipy.SpotifyException:
        print("Rate limit reached. Waiting 10s...")
        time.sleep(10)
        continue

    # Se guarda el progreso que se lleva cada 50 iteraciones y se imprime un mensaje para saber en qué fila va
    if i % 50 == 0:
        df.to_csv("progress.csv", index=False)
        print(f"Progreso guardado en la fila {i}")

    time.sleep(0.1)

df.to_csv("progress.csv", index=False)

#Consulta V2

Debido a que se tuvieron problemas por el límite se solicitudes con la primera versión del código, se tomó una pausa y se generó una versión de consulta diferente para evitar el problema. Esta segunda versión fue capaz de completar la base de datos, a excepción de las canciones que no contaban con una fecha de lanzamiento registrada por la API.

In [ ]:
import time
import pandas as pd
import spotipy

# Revisar si existe la columna 'release_date'
if 'release_date' not in df.columns:
    df['release_date'] = ""

# Filtrar para consultar solo los que no tienen la fecha aún
missing = df[df['release_date'].isna() | (df['release_date'] == "")]
track_ids = missing['track_id'].tolist()

print(f"Tracks por consultar: {len(track_ids)}")

# Procesar por lotes de 50 tracks
for i in range(0, len(track_ids), 50):
    batch = track_ids[i:i+50]

    try:
        results = sp.tracks(batch)
        for track in results['tracks']:
            if track is not None:
                track_id = track['id']
                release_date = track['album']['release_date']
                df.loc[df['track_id'] == track_id, 'release_date'] = release_date
    except spotipy.SpotifyException as e:
        if e.http_status == 429:  # error por limite
            retry_after = int(e.headers.get("Retry-After", 10))
            print(f"Rate limit alcanzado. Esperando {retry_after} segundos...")
            time.sleep(retry_after)
            continue
        else:
            raise  # otro tipo de error

    # Se guarda el progreso cada 500 tracks y se imprime para conocer en qué fila va
    if i % 500 == 0:
        df.to_csv("progress.csv", index=False)
        print(f"Progreso guardado hasta track {i}")

    time.sleep(0.2)  # se hace una pausa entre los lotes

df.to_csv("progress.csv", index=False)
print("Proceso finalizado y guardado.")


Tracks por consultar: 90796
Progreso guardado hasta track 0
Progreso guardado hasta track 500
Progreso guardado hasta track 1000
Progreso guardado hasta track 1500
Progreso guardado hasta track 2000
Progreso guardado hasta track 2500
Progreso guardado hasta track 3000
Progreso guardado hasta track 3500
Progreso guardado hasta track 4000
Progreso guardado hasta track 4500
Progreso guardado hasta track 5000
Progreso guardado hasta track 5500
Progreso guardado hasta track 6000
Progreso guardado hasta track 6500
Progreso guardado hasta track 7000
Progreso guardado hasta track 7500
Progreso guardado hasta track 8000
Progreso guardado hasta track 8500
Progreso guardado hasta track 9000
Progreso guardado hasta track 9500
Progreso guardado hasta track 10000
Progreso guardado hasta track 10500
Progreso guardado hasta track 11000
Progreso guardado hasta track 11500
Progreso guardado hasta track 12000
Progreso guardado hasta track 12500
Progreso guardado hasta track 13000
Progreso guardado hasta 

Se lee el nuevo dataset con la columna de fecha

In [ ]:
data = pd.read_csv('progress (1).csv')
data.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,release_date
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,2022-04-08
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,2021-04-30
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,2021-03-17
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,2018-08-10
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,2017-02-03


In [ ]:
data.tail()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,release_date
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.235,...,1,0.0422,0.640,0.928,0.0863,0.0339,125.995,5,world-music,2018-03-01
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.117,...,0,0.0401,0.994,0.976,0.1050,0.0350,85.239,4,world-music,2018-03-01
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.329,...,0,0.0420,0.867,0.000,0.0839,0.7430,132.378,4,world-music,1998-10-19
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.506,...,1,0.0297,0.381,0.000,0.2700,0.4130,135.960,4,world-music,1992
113999,113999,2hETkH7cOfqmz3LqZDHZf5,Cesária Evora,Miss Perfumado,Barbincor,22,241826,False,0.526,0.487,...,0,0.0725,0.681,0.000,0.0893,0.7080,79.198,4,world-music,1992-10-21


Se revisan valores nulos

In [ ]:
data.isna().sum()

,0
Unnamed: 0,0
track_id,0
artists,1
album_name,1
track_name,1
popularity,0
duration_ms,0
explicit,0
danceability,0
energy,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

Como nos percatamos de que la fecha de lanzamiento no tenía un formato estandarizado, decidimos solamente conservar el año.

In [ ]:
data['release_year'] = data['release_date'].astype(str).str[:4]

Imprimimos los valores que tomó la columna para ver que tengan un comportamiento lógico. Se observa que existen filas que tienen el valor 0000, esto se tratará posteriormente en la limpieza de datos.

In [ ]:
data['release_year'].unique()

array(['2022', '2021', '2018', '2017', '2014', '2008', '2015', '2012',
       '2009', '2020', '2019', '2010', '2013', '2007', '2016', '2011',
       '2000', '2005', '2024', '2006', '2003', '2002', '2004', '2001',
       '1969', '2023', '1997', '1998', '1995', '1993', '1962', '1958',
       '1983', '1966', '1978', '1972', '1967', '1992', '1971', '1987',
       '1959', '2025', '1989', '1976', '1986', '1927', '1991', '1990',
       '1984', '1973', '1999', '1994', '1996', '1974', '1979', '1977',
       '1982', '1970', '1980', '1968', '1988', '1985', '0000', '1975',
       '1981', '1957', '1965', '1960', '1961', '1963', '1955', '1964',
       '1951', '1956', '1950', '1954', '1953', '1900', '1935', '1952',
       '1949', '1941', '1913', '1923', '1946', '1945', '1934', '1905',
       '1931', '1944', '1929', '1939', '1943', '1938', '1925', '1926',
       '1948', '1930', '1928', '1936', '1940', '1937'], dtype=object)

In [ ]:
data['release_year'] = pd.to_numeric(data['release_year'])

In [ ]:
data.drop(columns=['release_date', 'Unnamed: 0'], inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_id          114000 non-null  object 
 1   artists           113999 non-null  object 
 2   album_name        113999 non-null  object 
 3   track_name        113999 non-null  object 
 4   popularity        114000 non-null  int64  
 5   duration_ms       114000 non-null  int64  
 6   explicit          114000 non-null  bool   
 7   danceability      114000 non-null  float64
 8   energy            114000 non-null  float64
 9   key               114000 non-null  int64  
 10  loudness          114000 non-null  float64
 11  mode              114000 non-null  int64  
 12  speechiness       114000 non-null  float64
 13  acousticness      114000 non-null  float64
 14  instrumentalness  114000 non-null  float64
 15  liveness          114000 non-null  float64
 16  valence           11

Se guarda el dataset con nueva información que posteriormente será sometido a limpieza y preprocesamiento.

In [ ]:
data.to_csv("spotify_dataset_mod.csv", index=False)